In [1]:
import cv2
import numpy as np
import time
import sys
from omnicv import fisheyeImgConv
import imutils
from matplotlib import pyplot as plt
import easyocr
import cv2

reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture('sample2.mp4')

# Set the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output3.mp4', cv2.VideoWriter_fourcc(*'H264'), 7, (frame_width, frame_height))

In [2]:
from videoprops import get_video_properties

props = get_video_properties('sample2.mp4')

print(f'''
Codec: {props['codec_name']}
Resolution: {props['width']}×{props['height']}
Aspect ratio: {props['display_aspect_ratio']}
Frame rate: {props['avg_frame_rate']}
''')


Codec: h264
Resolution: 7680×3840
Aspect ratio: 2:1
Frame rate: 285570000/40795531



In [ ]:
k=0
i=0
# Loop through each frame of the video
while cap.isOpened(): #i<100 for rerading 100 frames
    i+=1
    
    # Read the frame
    ret, frame = cap.read()
    
    # Check if the frame was successfully read
    if not ret:
        break
    
    #Code to see the frame
    #frame = cv2.resize(frame)#, None, fx=0.25, fy=0.25 
    
    inShape = frame.shape[:2]
    mapper = fisheyeImgConv()

    #Get the cubemap
    cubemap = mapper.equirect2cubemap(frame, side=3840, dice=True) #side of the cube is width/4 of 360(equirectangular) frame
    

    #print("Input shape", frame.shape)
    #print("Output shape", cubemap.shape)
    
    #See the cubemap
    #ex = cv2.resize(cubemap, None, fx=0.1, fy=0.1) 
    #cv2.imshow("cubemap", ex)
    
    #Code to get the faces of the cubemap
    #face0 face at the bottom
    #face5 rightmost face
    #face4 second last from rightmost face
    #face3 Centre face
    #face1 top face 
    #face 2 leftmost face
    height, width = cubemap.shape[:2]
    face_height = height // 3
    face_width = width // 4

    face0 = cubemap[face_height*2:face_height*3, face_width:face_width*2]
    face1 = cubemap[face_height:face_height*2, :face_width]#face_height*2:face_height*3, face_width*2:face_width*3]
    face2 = cubemap[:face_height, face_width:face_width*2]#face_height*2:face_height*3, face_width*3:]
    face3 = cubemap[face_height:face_height*2, face_width:face_width*2]
    face4 = cubemap[face_height:face_height*2, face_width*2:face_width*3]
    face5 = cubemap[face_height:face_height*2, face_width*3:]
    
    
    #not including face 0 as its the bottom face with 360 logo
    faces = [face1,face2,face3,face4,face5] 
    
    for j,face in enumerate(faces):
        try:
        #Face recognition and license plate detection and blurring code should go here 
            img = face.copy()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
            edged = cv2.Canny(bfilter, 30, 200) #Edge detection

            keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(keypoints)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

            location = None
            for contour in contours:
                approx = cv2.approxPolyDP(contour, 10, True)
                if len(approx) == 4:
                    location = approx
                    break
                    
            mask = np.zeros(gray.shape, np.uint8)
            new_image = cv2.drawContours(mask, [location], 0,255, -1)
            new_image = cv2.bitwise_and(img, img, mask=mask)

            (x,y) = np.where(mask==255)
            (x1, y1) = (np.min(x), np.min(y))
            (x2, y2) = (np.max(x), np.max(y))
            cropped_image = gray[x1:x2+1, y1:y2+1]

            reader = easyocr.Reader(['en'])
            result = reader.readtext(cropped_image)

            if(len(result)>0):
                text = result[0][-2]
                #If the text is readable then only blur the license plate 
                location = np.array([location], np.int32)
                points = location.reshape(4,2)
                xx,yy = np.split(points, [-1], axis=1)

                (xx1, xx2) = (np.min(xx), np.max(xx))
                (yy1, yy2) = (np.min(yy), np.max(yy))

                cropped_image = img[yy1:yy2, xx1:xx2]

                blurred_img = cv2.GaussianBlur(img,(101,101),30)

                #mask
                mask = np.zeros(face.shape,np.uint8)
                cv2.fillPoly(mask, location, (255,255,255))

                mask_inverse = np.ones(mask.shape).astype(np.uint8) *255 - mask

                #combine
                blur_result = cv2.bitwise_and(blurred_img, mask) + cv2.bitwise_and(img, mask_inverse)
                face = blur_result
                #saving the cue face when it blurs it
                cv2.imwrite("r"+str(k)+".jpg", blur_result)
                k+=1
                faces[j] = blur_result
            
        except Exception as e:
            #print("Not found")
            pass
                
        
    #storing the faces modfications back to cube map 
    cubemap[face_height:face_height*2, :face_width] = faces[0]
    cubemap[:face_height, face_width:face_width*2] = faces[1]
    cubemap[face_height:face_height*2, face_width:face_width*2] = faces[2]
    cubemap[face_height:face_height*2, face_width*2:face_width*3] = faces[3]
    cubemap[face_height:face_height*2, face_width*3:] = faces[4]
    
    #outshape of the output video
    outShape = [3840, 7680]
    inShape = cubemap.shape[:2]
    
    #This could be commented out 
    mapper = fisheyeImgConv()

    #converting cubmap to equirectangular
    equirect = mapper.cubemap2equirect(cubemap, outShape)
    
    #print("reoutput shape", equirect.shape)
    
    out.write(equirect)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
out.release()

# Close all windows
cv2.destroyAllWindows()